# Context managers and the with statement

The **`with`** statement in Python is a useful construct for properly managing external resources in your programs. 

It allows you to take advantage of **context managers** to automatically handle the setup and teardown phases whenever you’re dealing with external resources or with operations that require those phases.

In Python, you can use two general approaches to deal with resource management. You can wrap your code in:

    1) A try … finally construct
    2) A with construct
    
The second approach provides a straightforward way to use setup and teardown code, but you’ll have the limitation that the **`with`** statement only works with **context managers**. 

## The try … finally Approach

Working with files is probably the most common example of resource management in programming. 

In Python, you can use a `try … finally`statement to handle opening and closing files properly.

The `finally` clause will guarantee that file is properly closed, even if an exception occurs during the call to `write()` in the `try` clause.

Note that `open` should appear outside of the `try`. 

If `open` itself raises an exception, the file is not opened and does not need to be closed. 

Also, if open raises an exception its result is not assigned to `f` and it is an error to call `f.close()`.

In [2]:
f = open("hello.txt", "w")

try:
    f.write("Hello, World!")
finally:
    # Make sure to close the file after using it
    f.close()

13

The `try` block in the above example can potentially raise exceptions, such as `AttributeError` or `NameError`. 

You can handle those exceptions in an `except` clause like this:

In [3]:
f = open("hello.txt", "w")

try:
    f.write("Hello, World!")
except Exception as e:
    print(f"An error occurred while writing to the file: {e}")
finally:
    # Make sure to close the file after using it
    f.close()

13

## The with Statement

The Python **`with`** statement creates a runtime context that is used to run a group of statements under the control of a **context manager**. 

Compared to traditional `try … finally` constructs, the `with` statement can make your code clearer, safer, and more reusable.

To write a **with** statement, you need to use the following general syntax:

`with expression as var:
    # do something with var`
	
The expression after `with` must return a ***context manager object***, i.e. an object that implements the ***context management protocol***. 

This protocol consists of two special methods:

1. **`__enter__()`** is called by the with statement to enter the runtime context. `__enter__()`, typically provides the setup code

2. **`__exit__()`** is called when the execution leaves the with code block. `__exit__()` typically provides the teardown logic or cleanup code, such as calling `close()` on an open file object.
The `__exit__()` method takes 3 arguments (`exc_type`, `exc_val` , `exc_tb`). These are used to pass information about a potential exception that could happen within the `with` statements.
When there’s an exception if the `__exit__()` method returns `True` then that means it has been handled so it is not raised. If it returns anything else then it is raised by `with`.

The **`as`** specifier is optional. If you provide a ***`var`*** with **`as`**, then the value returned from the invocation of the special method `__enter__()` is bound to that variable.

**Note**: Some context managers return `None` from `__enter__()` because they have no useful object to give back to the caller. In these cases, specifying a var makes no sense.

Here’s how the **`with`** statement proceeds when Python runs into it:

- The `with` statement calls expression to obtain a context manager.
- The `with` statement calls `__enter__()` on the context manager and bind its return value to `var` if provided.
- The `with` statement executes the `with` code block.
- The `with` statement calls `__exit__()` on the context manager when the `with` code block finishes.

The previous file example can be implemented like that:

In [ ]:
with open("hello.txt", mode="w") as f:
    f.write("Hello, World!")

### Exceptions in with block

The `with` block in the above example can potentially raise exceptions, such as `AttributeError` or `NameError`. 

You can handle those exceptions in an `except` clause like this:

In [ ]:
try:
    with open("hello.txt", mode="w") as f :
        f.write("Hello, World!")
except Exception as ex: 
    print(f"Exception raised {ex}")

If the error has to do with opening the file (for instance, if the file doesn't exist), it will be raised by the call to open itself, not by `__enter__()`. 

In this case you can catch it by separating the open call from the with block:

In [ ]:
try:
    file = open("hello.txt", mode="w")
except Exception as ex: 
    print(f"Exception raised: {ex}")
else:
    with file:
        file.write("Hello, World!")

### How to deal with multiple context managers

You can supply `with` with any number of context managers separated by commas:

In [3]:
with open("hello.txt", mode="w") as outputF, \
     open("hello.txt") as imputF:
    outputF.write("Hello, World!")
    outputF.flush()
    imputF.readline()

13

'Hello, World!'

**Python 3.10** introduces a new feature: using enclosing parentheses for continuation across multiple lines in context managers is now supported.

In [ ]:
with (open("hello.txt", mode="w") as outputF, 
     open("hello.txt") as inputF):
    outputF.write("Hello, World!")
    outputF.flush()
    inputF.readline()

## How To Implement a Context Manager

There are two ways to implement a context manager. 

1. The first one is defining a class with implementations for the `__enter__()` and `__exit__()` methods. 
2. The second one is by creating a generator and using the `contextlib.contextmanager` decorator.

### An example using a custom class

In [2]:
class CustomFileOpen:
    """Custom context manager for opening files."""

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        print("__enter__ is executed")
        self.f = open(self.filename, self.mode)
        return self.f

    def __exit__(self, *args):
        print("__exit__ is executed")
        self.f.close()

with CustomFileOpen("file.txt", "w") as f:
    f.write("contents go here")

__enter__ is executed


16

__exit__ is executed


### The @contextmanager decorator 

Creating context managers by writing a class with `__enter__()` and `__exit__()` methods, is not difficult. 

However, you can achieve better brevity by defining them using **`contextlib.contextmanager`** decorator. 

This decorator converts a ***generator function*** into a context manager.

The code within `CustomFileOpen` will run until we reach `yield`. At that point, we will go within the `with` statement. 

If any exceptions happen they can be handled as we are within a `try-except-finally` block.

You can think of what happens in the except and finally blocks as equivalent to what happens in `__exit__()`.

In [ ]:
from contextlib import contextmanager

@contextmanager
def CustomFileOpen(filename, method):
    """Custom context manager for opening a file."""

    f = open(filename, method)
    try:
        yield f

    finally:
        f.close()
        
with CustomFileOpen("hello.txt", "w") as f:
    f.write("Hello, World!")

In [6]:
from contextlib import contextmanager
from time import time

@contextmanager
def runtime(description):

    print(description)
    start_time = time()
    try:
        yield
    finally:
        end_time = time()
        run_time = end_time - start_time
        print(f"The function took {run_time} seconds to run.")
        
@runtime("This function create a simple file")
def custom_file_write(filename, mode, content):
    with open(filename, mode) as f:
        f.write(content)
        
print(custom_file_write("file.txt", "w", "Hello World"))

This function create a simple file
The function took 0.0 seconds to run.
None
